In [1]:
# FauxFoe is being produced by myself and Megan Marelli, a colleague and fellow CJS student. 

# The premise is quite simple. We will leverage Google's new ClaimReview to monitor Twitter. First and foremost, 
# if a Twitter user publishes a link that has been subjected to Google's ClaimReview, FauxFoe will comment on that 
# tweet or DM the user responsible for the tweet notifying them that the claim has been reviewed. 

# As FauxFoe continues to grow, we plan to parse each tweet so that we can address users that rely on 'fake news' 
# for information rather than for direct posts. For instance, if somebody uses a ClaimReviewed fact in a Twitter 
# argument, but does not post the exact link, FauxFoe will be able to address that claim. 

# Alas, for now, FauxFoe is in its infancy. Below is some of the base code that will be used to operate FauxFoe. 

In [2]:
# The guideliens to the metadata can be found here... 
# http://schema.org/ClaimReview

In [3]:
import requests
# Requests documentation: http://docs.python-requests.org/en/master/

url = 'http://feeds.washingtonpost.com/rss/rss_fact-checker'
# As an example of FauxFoe's leveraging ClaimReview, we will pull the rss feed from the Washington Post's Fact 
# Checker, which includes articles that utilize Google's ClaimReview. 

r = requests.get(url)

#print r.text
# Below is the HTML text from the WaPo Fact Checker RSS feed. 

In [4]:
from bs4 import BeautifulSoup
# BeautifulSoup documentation: https://www.crummy.com/software/BeautifulSoup/bs4/doc/

soup = BeautifulSoup(r.text)

#print soup

//anaconda/lib/python2.7/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 174 of the file //anaconda/lib/python2.7/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [5]:
url_list = []
# Create a new, empty array in which we can store the URL's for each of the ClaimReviewed articles featured on the 
# WaPo Fact Checker site. 

for a in soup.find_all('a', href=True): 
    #print "Found the URL:", a['href']
    url_list.append(a['href'])

In [6]:
#url_list

In [7]:
for i in url_list: 
# For every URL...
    x = requests.get(i)
    soup = BeautifulSoup(x.text)
    #print(soup)
    # Print the 'soup' (aka 'BeautifulSoup the URL')

In [8]:
# We will now pull aspects of the ClaimReview micro data (which can be found at schema.org) for each URL so that we
# can later put that data into a dataframe (it'll all make sense in a moment!)

x = requests.get(i)
soup = BeautifulSoup(x.text)
l = (soup(itemprop="claimReviewed"))
m = (soup(itemprop="ratingValue"))
n = (soup(itemprop="datePublished"))
o = str(i)

In [9]:
#print l
#print m 
#print n
#print o

In [10]:
# To explore the rating scale, visit: https://developers.google.com/search/docs/data-types/factcheck

In [11]:
import pandas as pd
from pandas import DataFrame

In [12]:
#print "claimReviewed: " + l[0].text
#print "ratingValue: " + m[0].text
#print "datePublished: " + n[0].text
#print "url: " + o

In [13]:
column_names = ["claimReviewed","datePublished","ratingValue","url"]
# Generate the column names for our data frame... 

rows = []

no_claimreview = []
# Generate an empty array titled 'rows' (which we will import our ClaimReview data into in a moment)

for i in url_list: 
    x = requests.get(i)
    soup = BeautifulSoup(x.text)
    
    l = (soup(itemprop="claimReviewed"))
    m = (soup(itemprop="ratingValue"))
    n = (soup(itemprop="datePublished"))
    o = str(i)
    # Note that this is the same code as before, only now we are importing it into the data frame! 
    if l and m and n and o: 
        rows.append([l[0].text,n[0].text,m[0].text,o]) 
    else: 
        no_claimreview.append(i)

df = DataFrame(rows,columns=column_names)

print no_claimreview

['http://www.washingtonpost.com/blogs/fact-checker/wp/2017/05/01/president-trumps-first-100-days-the-fact-check-tally/']


In [14]:
pd.options.display.max_colwidth = 300

df

,claimReviewed,datePublished,ratingValue,url
0,"""One estimate said that millions would pay more, and the rich do extremely well.”",May 2 at 3:00 AM,-1,http://www.washingtonpost.com/blogs/fact-checker/wp/2017/05/02/schumers-claim-that-millions-would-pay-more-under-trumps-tax-plan/
1,"""A city that prohibits its officials from providing information to federal immigration authorities — a sanctuary city — is violating the law. Sanctuary cities, like San Francisco, block their jails from turning over criminal aliens to federal authorities for deportation.""",April 28,3,http://www.washingtonpost.com/blogs/fact-checker/wp/2017/04/28/the-white-houses-claim-that-sanctuary-cities-are-violating-the-law/
2,"“In over half of American households, the single largest expense is child care, even exceeding the cost of housing.”",April 27,-1,http://www.washingtonpost.com/blogs/fact-checker/wp/2017/04/27/ivanka-trumps-claim-that-for-half-of-u-s-families-the-single-largest-expense-is-child-care/
3,“An undocumented immigrant is not a criminal.”,April 26,3,http://www.washingtonpost.com/blogs/fact-checker/wp/2017/04/26/sen-kamala-harriss-claim-that-an-undocumented-immigrant-is-not-a-criminal/
4,"""In office, President Trump has accomplished more in his first 100 days than any other President since Franklin Roosevelt.”",April 25,-1,http://www.washingtonpost.com/blogs/fact-checker/wp/2017/04/25/the-white-house-boo-boo-in-counting-roosevelts-executive-orders/
5,"""President Xi [of China], from the time I took office, they have not been currency manipulators.”",April 25,-1,http://www.washingtonpost.com/blogs/fact-checker/wp/2017/04/25/fact-checking-president-trumps-pinocchio-laden-associated-press-interview/
6,"""Somebody, yeah, somebody put out the concept of a hundred-day plan.""",April 24,5,http://www.washingtonpost.com/blogs/fact-checker/wp/2017/04/24/who-was-the-somebody-who-issued-a-100-day-plan-for-president-trump/
7,"Wendy Sherman ""was the architect of the North Korean nuclear deal""",April 24,4,http://www.washingtonpost.com/blogs/fact-checker/wp/2017/04/24/the-facile-claim-that-the-iran-negotiator-was-the-architect-of-the-north-korean-nuclear-deal/
8,"The American Health Care Act ""will reduce premiums by 10 percent. We would actually bring costs down.”",April 21,-1,http://www.washingtonpost.com/blogs/fact-checker/wp/2017/04/21/how-is-medicare-affected-by-the-house-gop-health-plan/


In [15]:
CONSUMER_KEY = "TKTKTKTKTK"
CONSUMER_SECRET = "TKTKTKTKTK"
ACCESS_TOKEN = "TKTKTKTKTK"
ACCESS_TOKEN_SECRET = "TKTKTKTKTK"

In [16]:
from tweepy import OAuthHandler, API
# Tweepy documentation: http://docs.tweepy.org/en/v3.5.0/

# Setup the authentication
auth = OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

# Create an object we will use to communicate with the Twitter API
api = API(auth)

In [17]:
from tweepy import Cursor
# A brief introduction to Tweepy's Cursor: http://docs.tweepy.org/en/v3.5.0/cursor_tutorial.html#introduction

for query in df["claimReviewed"]: 

    ids = []
    texts = []
    times = []
    retweets = []
    screen_names = []
    followers_counts = []
    friends_counts = []

# Only iterate through the first 3 pages (for now!)
    for page in Cursor(api.search, q=query, result_type='recent', count=100, until="2017-04-26").pages(170):
    
        for tweet in page:

            ids.append(tweet.id)
            texts.append(tweet.text)
            times.append(tweet.created_at)
            retweets.append(tweet.retweet_count)
            screen_names.append(tweet.user.screen_name)
            friends_counts.append(tweet.user.friends_count)
            followers_counts.append(tweet.user.followers_count)
            
            print ids
            print "----- ----- -----"
            
            
            df2 = DataFrame({"id":ids,"text":texts,"time":times,"retweet":retweets,"screen_name":screen_names,"friends_count":friends_counts,"followers_count":followers_counts})

[857014844255596545]
----- ----- -----
[857014844255596545, 856991409689309184]
----- ----- -----
[857014844255596545, 856991409689309184, 856969153055518720]
----- ----- -----
[857014844255596545, 856991409689309184, 856969153055518720, 856965904621486080]
----- ----- -----
[857014844255596545, 856991409689309184, 856969153055518720, 856965904621486080, 856963360738246656]
----- ----- -----
[857014844255596545, 856991409689309184, 856969153055518720, 856965904621486080, 856963360738246656, 856959834763034624]
----- ----- -----
[857014844255596545, 856991409689309184, 856969153055518720, 856965904621486080, 856963360738246656, 856959834763034624, 856957491959279616]
----- ----- -----
[857014844255596545, 856991409689309184, 856969153055518720, 856965904621486080, 856963360738246656, 856959834763034624, 856957491959279616, 856957394651193344]
----- ----- -----
[857014844255596545, 856991409689309184, 856969153055518720, 856965904621486080, 856963360738246656, 856959834763034624, 8569574

In [18]:
df2

,followers_count,friends_count,id,retweet,screen_name,text,time
0,251,846,856984796416823296,0,tweetheartsue,"""It wasn't me!""\n\n(""Somebody, yeah, somebody put out the concept of a hundred-day plan."" Oh right, that would be...y… https://t.co/DR9HYNeTtF",2017-04-25 21:34:32
1,35,73,856686671248539648,1,DSDub22,"RT @sircosmokramer: @JackiSchechner @SMShow TRUMP: Somebody, yeah, somebody put out the concept of a hundred-day plan. But yeah. Well, I'm…",2017-04-25 01:49:53
2,16948,15632,856630088225484800,0,YngLiberalGirl,"President Donald Trump: Somebody, yeah, somebody put out the concept of a hundred-day plan. | Four Pinocchios https://t.co/4wmK2Ryd55",2017-04-24 22:05:03
3,376,29,856563425132003328,0,viralnewsx,"President Donald Trump: Somebody, yeah, somebody put out the concept of a hundred-day plan. | Four Pinocchios… https://t.co/kosVVSJcoS",2017-04-24 17:40:09
4,364,1187,856544821980299264,0,KTSavage,"TRUMP: “Somebody, yeah, somebody put out the concept of a hundred-day plan. But yeah. Well, I’m mostly there on mos… https://t.co/GGgJToP4b5",2017-04-24 16:26:14
5,299,107,856523848912703492,0,frakincaprica,"Trump Candidate: ""I will do xyz in 100 days""\n\nTrump AP:\n\n""Somebody, yeah, somebody put out the concept of a hundred-day plan.""\n\n😂😂\n\n#TrumpA",2017-04-24 15:02:53
6,114,83,856506148450467840,1,sircosmokramer,"@JackiSchechner @SMShow TRUMP: Somebody, yeah, somebody put out the concept of a hundred-day plan. But yeah. Well,… https://t.co/RFw7N8myKV",2017-04-24 13:52:33
7,6179,1198,856474391189835776,0,danfagin,"""Somebody, yeah, somebody put out the concept of a hundred-day plan.""\n\nSomebody did all right, yeah.\n\nhttps://t.co/I523oH8RTF",2017-04-24 11:46:22
8,1562,4665,856346865599754240,1,AmStillNotYou,"RT @AmStillNotYou: @williamlegate @realDonaldTrump c/DJT: ""Somebody, yeah, somebody put out the concept of a hundred-day plan. But yeah. We…",2017-04-24 03:19:37
9,1562,4665,856346837644709888,1,AmStillNotYou,"@williamlegate @realDonaldTrump c/DJT: ""Somebody, yeah, somebody put out the concept of a hundred-day plan. But yea… https://t.co/2vUmkB81Dn",2017-04-24 03:19:30


In [19]:
bigdata = pd.concat([df, df2], ignore_index=True)


In [20]:
bigdata

,claimReviewed,datePublished,followers_count,friends_count,id,ratingValue,retweet,screen_name,text,time,url
0,"""One estimate said that millions would pay more, and the rich do extremely well.”",May 2 at 3:00 AM,NaN,NaN,NaN,-1,NaN,NaN,NaN,NaT,http://www.washingtonpost.com/blogs/fact-checker/wp/2017/05/02/schumers-claim-that-millions-would-pay-more-under-trumps-tax-plan/
1,"""A city that prohibits its officials from providing information to federal immigration authorities — a sanctuary city — is violating the law. Sanctuary cities, like San Francisco, block their jails from turning over criminal aliens to federal authorities for deportation.""",April 28,NaN,NaN,NaN,3,NaN,NaN,NaN,NaT,http://www.washingtonpost.com/blogs/fact-checker/wp/2017/04/28/the-white-houses-claim-that-sanctuary-cities-are-violating-the-law/
2,"“In over half of American households, the single largest expense is child care, even exceeding the cost of housing.”",April 27,NaN,NaN,NaN,-1,NaN,NaN,NaN,NaT,http://www.washingtonpost.com/blogs/fact-checker/wp/2017/04/27/ivanka-trumps-claim-that-for-half-of-u-s-families-the-single-largest-expense-is-child-care/
3,“An undocumented immigrant is not a criminal.”,April 26,NaN,NaN,NaN,3,NaN,NaN,NaN,NaT,http://www.washingtonpost.com/blogs/fact-checker/wp/2017/04/26/sen-kamala-harriss-claim-that-an-undocumented-immigrant-is-not-a-criminal/
4,"""In office, President Trump has accomplished more in his first 100 days than any other President since Franklin Roosevelt.”",April 25,NaN,NaN,NaN,-1,NaN,NaN,NaN,NaT,http://www.washingtonpost.com/blogs/fact-checker/wp/2017/04/25/the-white-house-boo-boo-in-counting-roosevelts-executive-orders/
5,"""President Xi [of China], from the time I took office, they have not been currency manipulators.”",April 25,NaN,NaN,NaN,-1,NaN,NaN,NaN,NaT,http://www.washingtonpost.com/blogs/fact-checker/wp/2017/04/25/fact-checking-president-trumps-pinocchio-laden-associated-press-interview/
6,"""Somebody, yeah, somebody put out the concept of a hundred-day plan.""",April 24,NaN,NaN,NaN,5,NaN,NaN,NaN,NaT,http://www.washingtonpost.com/blogs/fact-checker/wp/2017/04/24/who-was-the-somebody-who-issued-a-100-day-plan-for-president-trump/
7,"Wendy Sherman ""was the architect of the North Korean nuclear deal""",April 24,NaN,NaN,NaN,4,NaN,NaN,NaN,NaT,http://www.washingtonpost.com/blogs/fact-checker/wp/2017/04/24/the-facile-claim-that-the-iran-negotiator-was-the-architect-of-the-north-korean-nuclear-deal/
8,"The American Health Care Act ""will reduce premiums by 10 percent. We would actually bring costs down.”",April 21,NaN,NaN,NaN,-1,NaN,NaN,NaN,NaT,http://www.washingtonpost.com/blogs/fact-checker/wp/2017/04/21/how-is-medicare-affected-by-the-house-gop-health-plan/
9,NaN,NaN,251.0,846.0,8.569848e+17,NaN,0.0,tweetheartsue,"""It wasn't me!""\n\n(""Somebody, yeah, somebody put out the concept of a hundred-day plan."" Oh right, that would be...y… https://t.co/DR9HYNeTtF",2017-04-25 21:34:32,NaN


In [21]:
for i in bigdata: 
    if [bigdata["ratingValue"] < 0]: 
        print "Hi, " + bigdata["screen_name"] + "it looks like Google has proven this false. Check WaPo.com for more."
        print "--- --- --- --- ---"

0                                                                                         NaN
1                                                                                         NaN
2                                                                                         NaN
3                                                                                         NaN
4                                                                                         NaN
5                                                                                         NaN
6                                                                                         NaN
7                                                                                         NaN
8                                                                                         NaN
9       Hi, tweetheartsueit looks like Google has proven this false. Check WaPo.com for more.
10            Hi, DSDub22it looks like Google has proven thi